In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator

import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.feature_extraction.text import CountVectorizer


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/all_discipline_data.pql"
data = pd.read_pickle(data_path)

In [5]:
data50 = data[data.PubYear == 1950]

In [6]:
data50_group = data50[["Label","title_without_stopwords"]].groupby(data50.Label)["title_without_stopwords"].apply(list)

data50_group = pd.DataFrame(data50_group)

In [7]:
data50_group["title_words"] = data50_group["title_without_stopwords"].apply(lambda x: " ".join(x))

In [8]:
data50_group.iloc[449]

title_without_stopwords    [diffraction pattern circular aperture measure...
title_words                diffraction pattern circular aperture measured...
Name: Physics, Applied, dtype: object

In [9]:
len(list(data50_group.index))

613

In [69]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(use_idf=True, min_df=5, stop_words='english')
# tf_idf_matrix = vectorizer.fit_transform(data50_group.title_words)

# num_samples, num_features = tf_idf_matrix.shape
# print("num_samples", num_samples)
# print("num_features", num_features)

num_samples 613
num_features 23816


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=2, max_df=1.0)
vectorized_words = vectorizer.fit_transform(data50_group.title_words)

In [49]:
#1. Document Frequency - Number of disciplines in which the word has appeared

vectorized_wordarray = vectorized_words.toarray()
document_frequency = (vectorized_wordarray != 0).sum(0).tolist()
document_frequency_log =  [np.log(x) for x in document_frequency]
#2. Term Frequency - Number of times in which a word appears in a document
term_frequency = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequency_log = [np.log(x) for x in term_frequency]

In [50]:
count_vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

In [41]:
#tf_idf_vocab = {value:key for key,value in vectorizer.vocabulary_}
#tf_idf_vocab
#tfidf_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
#words = [tfidf_rev_dict[i] for i in range(len(vectorizer.idf_))]

In [42]:
# consider one discipline - Applied Physics index 449

# get tf scores of all the words of applied physics
#tf_applied_physics = tf_idf_matrix[450:451,:].todense().reshape(-1,).tolist()[0]
#idf_words = vectorizer.idf_

#tf_idf_physics = [(tf_applied_physics[i], idf_words[i]) for i in range(len(idf_words))]

trace0 = go.Scatter(
    x = document_frequency_log,
    y = term_frequency_log,
    mode = 'markers',
    text = words,
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)


layout= go.Layout(
    title= 'Document Frequency - Term Frequency plot ',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Log Document Frequency (Number of disciplines in which the word occurs)',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Log Term Frequency (Count of the word in corpus)',
        ticklen= 5,
        gridwidth= 2,
    ),
    
    showlegend= False
)

plotdata = [trace0]


fig= go.Figure(data=plotdata, layout=layout)
py.iplot(fig)

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:


The draw time for this plot will be slow for clients without much RAM.


/usr/local/lib/python3.6/dist-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



Let us pick a couple of "good" words (words which occur in multiple disciplines and are whose log tf are not too high) for analysis

__Words:  neutron, influenza, ketones, protons, tuberculin__

In [69]:
# which all disciples have neutron in them ?
# lets find the indices


# index of the word - 
index = 37037
words.index("neutron")
count_rev_dict[index]
counts = vectorized_wordarray[:,index]

discipline_indices = [discipline_index for discipline_index in range(len(counts.tolist())) if counts[discipline_index] != 0]

9

__Which disciplines are those?__

In [73]:
[list(data50_group.index)[index] for index in discipline_indices]

['Chemistry',
 'Chemistry, Multidisciplinary',
 'Chemistry, Physical',
 'Instruments & Instrumentation',
 'Life Sciences & Biomedicine',
 'Multidisciplinary Sciences',
 'Physical Sciences',
 'Physics',
 'Physics, Applied',
 'Physics, Atomic, Molecular & Chemical',
 'Physics, Multidisciplinary',
 'Physics, Nuclear',
 'Radiology, Nuclear Medicine & Medical Imaging',
 'Science & Technology',
 'Science & Technology - Other Topics',
 'Technology']